In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split



In [3]:
ml_data = pd.read_csv("ml_data.csv")
ml_data=ml_data.astype("int64")
X = ml_data.drop(columns=['available_bikes'])  # Features
y = ml_data['available_bikes']  # Target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

class BikeDataset(Dataset):
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, row):
        feature = torch.tensor(self.features.iloc[row].values, dtype=torch.int64)
        target = torch.tensor(self.targets.iloc[row], dtype=torch.int64).unsqueeze(-1)
        return feature, target
training_data = BikeDataset(X_train, y_train)



test_data = BikeDataset(X_test, y_test)

In [4]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [5]:
training_data.train_data.shape

c:\Users\carlo\anaconda3\envs\Pytorch\Lib\site-packages\torchvision\datasets\mnist.py:75: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


torch.Size([60000, 28, 28])

In [6]:
training_data.targets.shape

torch.Size([60000])

In [7]:
training_data.train_data.shape

torch.Size([60000, 28, 28])

In [8]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [9]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [10]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [11]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [7]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.300337  [   64/60000]
loss: 2.289714  [ 6464/60000]
loss: 2.275207  [12864/60000]
loss: 2.264568  [19264/60000]
loss: 2.252731  [25664/60000]
loss: 2.227719  [32064/60000]
loss: 2.229867  [38464/60000]
loss: 2.200910  [44864/60000]
loss: 2.194270  [51264/60000]
loss: 2.163631  [57664/60000]
Test Error: 
 Accuracy: 48.2%, Avg loss: 2.157655 

Epoch 2
-------------------------------
loss: 2.166683  [   64/60000]
loss: 2.157814  [ 6464/60000]
loss: 2.108148  [12864/60000]
loss: 2.119241  [19264/60000]
loss: 2.070658  [25664/60000]
loss: 2.015189  [32064/60000]
loss: 2.039468  [38464/60000]
loss: 1.964051  [44864/60000]
loss: 1.968539  [51264/60000]
loss: 1.898482  [57664/60000]
Test Error: 
 Accuracy: 60.0%, Avg loss: 1.895671 

Epoch 3
-------------------------------
loss: 1.928028  [   64/60000]
loss: 1.899532  [ 6464/60000]
loss: 1.784812  [12864/60000]
loss: 1.821834  [19264/60000]
loss: 1.714719  [25664/60000]
loss: 1.665185  [32064/600

In [ ]:
# Define model
class model_simple(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear = nn.Sequential(
            nn.Linear(135, 20)
            

        )

    def forward(self, x):
        x = self.flatten(x)

        print("Input shape:", x.shape)  # Print input tensor shape

        y_pred = self.linear(x)
        print("Output shape:", y_pred.shape)
        return y_pred

model = model_simple()

loss_fn = nn.MSELoss()

optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
print(model)